In [ ]:
from schema import mine_schema
from new_schema import new_schema
from fuzzywuzzy import fuzz, process
from colnames import colnames
import pandas as pd

def clear_schema():
    for k in new_schema:
        new_schema[k] = []

def update_new_schema():
    f = open('new_schema.py', 'w')
    f.write(f'new_schema = {new_schema}')
    f.close()

def find_matched_colname(target):
    found_match = False
    for key in new_schema:
        if fuzz.partial_ratio(target, key) >= 75:
            new_schema[key].append(target)
            found_match = True
    if found_match == False:
        new_schema['keep_it_as_it_is'].append(target)

In [ ]:
def find_close_match(target):
    matches = []
    for col in standard_colnames:
        if fuzz.ratio(target, col) > 80:
            matches.append(col)
    return matches    

In [ ]:
def find_match(colname):
    options = list(new_schema.items())
    # matches_count = 0
    matches = []
    for i in options:
        current_values = [i[0]] + i[1]
        for v in current_values:
            if fuzz.ratio(colname, v) >= 85:
                matches.append(i[0])
    return (colname, matches)

In [ ]:
f = open('matches.txt', 'w')
for c in colnames:
    f.write(f'{find_match(c)}\n')
f.close()

In [ ]:
clear_schema()
for c in colnames:
    find_matched_colname(c)
update_new_schema()

In [ ]:
find_match('company')

In [ ]:
# https://www.datacamp.com/community/tutorials/fuzzy-string-python
# make it flexible? If high yields no match, then try again
def find_best_matches(colname, scorer, score_cutoff):
    results_with_scores = process.extractBests(colname, list(new_schema.keys()), scorer=scorer, score_cutoff=score_cutoff)
    results_only = list((map(lambda result: result[0], results_with_scores)))
    return results_only

In [ ]:
scorers = {"fuzz.ratio": fuzz.ratio, "fuzz.token_set_ratio": fuzz.token_set_ratio, "fuzz.token_sort_ratio": fuzz.token_sort_ratio, "fuzz.partial_ratio": fuzz.partial_ratio, "fuzz.partial_token_set_ratio": fuzz.partial_token_set_ratio, "fuzz.partial_token_sort_ratio": fuzz.partial_token_sort_ratio}

In [ ]:
# write a function that keeps the score for each scorer
accuracy_scores = {"fuzz.ratio": 0, "fuzz.token_set_ratio": 0, "fuzz.token_sort_ratio": 0, "fuzz.partial_ratio": 0, "fuzz.partial_token_set_ratio": 0, "fuzz.partial_token_sort_ratio": 0}


In [ ]:
def compare_scorers(target, score_cutoff):
    res = ''
    res += f'Best matches for "{target.upper()}" at {score_cutoff}-point threshold\n\n'
    for s in scorers:
        res += f'{s}: {find_best_matches(target, scorer=scorers[s], score_cutoff=score_cutoff)}\n'
    res += f'{20*"="}\n\n'
    return res

In [ ]:
res_file = open('compare_fuzz_methods.txt', 'w')


In [ ]:
for c in colnames:
    res_file.write(f'{compare_scorers(c, 75)}')
res_file.close()

In [ ]:
# could run all methods on a word and then gather all the choices to present to users

def matches_from_all_methods(target, score_cutoff):
    res = []
    for s in scorers:
        res += find_best_matches(target, scorers[s], score_cutoff=score_cutoff)
    return list(set(res))

In [ ]:
def matches_for_all_colnames(score_cutoff):
    res_file = open('compare_fuzz_methods.txt', 'w')
    for col in colnames:
        res_file.write('\n')
        res_file.write(f'Matches for {col.upper()} at {score_cutoff}-point threshold\n\n')
        res_file.write(f'{col}: {matches_from_all_methods(col, score_cutoff)}\n')
        res_file.write(40*'=' + '\n')
    res_file.close()
        

In [ ]:
matches_for_all_colnames(60)